In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os.path as op
from pprint import pprint as pp
import scipy.stats
import seaborn as sns
import sys
pth = op.abspath("../")
print pth
sys.path.insert(0, pth)
import teigen
reload(teigen)
print teigen.__path__

/Users/mjirik/projects/teigen
Using QVariant and QString version 1
['/Users/mjirik/projects/teigen/teigen']


In [3]:
run_list = [
    # "compensation method",
    "compensation method 1 sphere",
    # "compensation method 10 spheres"
    # "compensation method 1 tube",
    # "resolution tube"
]

In [4]:
tg = teigen.tgmain.Teigen()
config = tg.get_default_config()

In [5]:
# pp(config)
config

OrderedDict([('generators',
              OrderedDict([('Voronoi tubes',
                            OrderedDict([('element_number', 30),
                                         ('uniform_radius_distribution', True),
                                         ('normal_radius_distribution', False),
                                         ('fixed_radius_distribution', False),
                                         ('radius_distribution_minimum', 2.0),
                                         ('radius_distribution_maximum', 10.0),
                                         ('radius_distribution_mean', 5.0),
                                         ('radius_distribution_standard_deviation',
                                          5.0),
                                         ('random_generator_seed', 0)])),
                           ('Gensei', OrderedDict([('n_objects', 10)])),
                           ('Continuous tubes',
                            OrderedDict([('element_number', 3

In [6]:
np.arange(0, 10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# Prepare multiple configs


In [7]:
configs = []
tg = teigen.tgmain.Teigen()

In [8]:
run_label = "tube length"
if run_label in run_list:
    for length in [5, 6, 7, 8, 10, 12, 14, 16, 20, 24, 28, 32, 40, 48]:
        for rnd in [101, 102, 103]:
            config = tg.get_default_config()
            config["generator_id"] = 3
            config["output"]["note"] = run_label
            gc = config["generators"]["Unconnected tubes"]
            gc["element_number"] = 10
            gc["orientation_anisotropic"] = False
            gc["length_distribution_mean"] = length
            gc["length_distribution_standard_deviation"] = 0.0
            gc["random_generator_seed"] = rnd
            config["postprocessing"]["measurement_resolution"] = 20
            config["appearance"]["skip_volume_generation"] = True
            configs.append(config)

In [9]:
run_label = "tube length 2"
if run_label in run_list:
    for length in [5, 10, 15, 20, 25, 30, 35, 40, 45]:
        for rnd in [101, 102, 103]:
            config = tg.get_default_config()
            config["generator_id"] = 3
            config["output"]["note"] = run_label
            gc = config["generators"]["Unconnected tubes"]
            gc["element_number"] = 10
            gc["orientation_anisotropic"] = False
            gc["length_distribution_mean"] = length
            gc["length_distribution_standard_deviation"] = 0.0
            gc["random_generator_seed"] = rnd
            config["postprocessing"]["measurement_resolution"] = 20
            config["appearance"]["skip_volume_generation"] = True
            configs.append(config)

# Vepsaná, opsaná, 

In [10]:
run_label = "compensation method"
if run_label in run_list:
    for resolution in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
        for rnd in [101]:
            for radius_method in ["inscribed", "circumscribed", "cylinder surface", "cylinder volume", "average"]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 10
                gc["orientation_anisotropic"] = False
                gc["uniform_radius_distribution"] = False
                gc["fixed_radius_distribution"] = True
                gc["radius_distribution_mean"] = 20
                gc["radius_distribution_standard_deviation"] = 0.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 20
                gc["length_distribution_standard_deviation"] = 0.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

### 1 sphere

In [11]:
run_label = "compensation method 1 sphere"
if run_label in run_list:
    for resolution in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
        for rnd in [101]:
            for radius_method in ["inscribed", "circumscribed", "cylinder surface", "cylinder volume", "average", 
                                  "cylinder volume + sphere compensation"]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 1
                gc["orientation_anisotropic"] = False
                gc["uniform_radius_distribution"] = False
                gc["fixed_radius_distribution"] = True
                gc["radius_distribution_mean"] = 20
                gc["radius_distribution_standard_deviation"] = 0.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 0.0
                gc["length_distribution_standard_deviation"] = 0.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

### 10 spheres

In [12]:
run_label = "compensation method 10 spheres"
if run_label in run_list:
    for resolution in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
        for rnd in [101]:
            for radius_method in ["inscribed", "circumscribed", "cylinder surface", "cylinder volume", "average"]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 10
                gc["uniform_radius_distribution"] = False
                gc["fixed_radius_distribution"] = True
                gc["orientation_anisotropic"] = False
                gc["radius_distribution_mean"] = 20
                gc["radius_distribution_standard_deviation"] = 0.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 0.0
                gc["length_distribution_standard_deviation"] = 0.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

In [13]:
run_label = "compensation method 1 tube"
if run_label in run_list:
    for resolution in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
        for rnd in [101]:
            for radius_method in ["cylinder surface", "cylinder volume", "cylinder volume + sphere compensation"]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 1
                gc["uniform_radius_distribution"] = False
                gc["fixed_radius_distribution"] = True
                gc["orientation_anisotropic"] = False
                gc["radius_distribution_mean"] = 20
                gc["radius_distribution_standard_deviation"] = 0.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 0.0
                gc["length_distribution_standard_deviation"] = 0.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

# Všehochuť

In [14]:
# run_label = "resolution shape"
# if run_label in run_list:
#     for resolution in [5, 6, 7, 8, 10, 12, 14, 16, 20, 24, 28, 32, 40, 48]:
#         for rnd in [101, 102, 103, 104, 105]:
#             config = tg.get_default_config()
#             config["generator_id"] = 3
#             config["output"]["note"] = run_label
#             gc = config["generators"]["Unconnected tubes"]
#             gc["element_number"] = 10
#             gc["orientation_anisotropic"] = False
#             gc["length_distribution_mean"] = 0.0
#             gc["length_distribution_standard_deviation"] = 0.0
#             gc["random_generator_seed"] = rnd
#             config["postprocessing"]["measurement_resolution"] = resolution
#             config["appearance"]["skip_volume_generation"] = True
#             configs.append(config)

run_label = "resolution tube"
if run_label in run_list:
    for resolution in [6, 7, 9, 11, 13, 15, 19, 23, 28, 33, 39, 50]:
        for rnd in [102, 103]:
            config = tg.get_default_config()
            config["generator_id"] = 3
            config["output"]["note"] = run_label
            gc = config["generators"]["Unconnected tubes"]
            gc["element_number"] = 10
            gc["orientation_anisotropic"] = False
            gc["length_distribution_mean"] = 40.0
            gc["length_distribution_standard_deviation"] = 2.0
            gc["random_generator_seed"] = rnd
            config["postprocessing"]["measurement_resolution"] = resolution
            config["appearance"]["skip_volume_generation"] = True
            config["measurement"]["polygon_radius_selection_method"] = "cylinder volume + sphere compensation"
            configs.append(config)


# Run multiple configs

In [15]:
for i, cfg in enumerate(configs):
    
    tg = teigen.tgmain.Teigen()
    # config = tg.get_default_config()
    tg.update_config(**cfg)
    print (str(i) + " / " + str(len(configs)))  
    print(gc["random_generator_seed"])
    print(config["postprocessing"]["measurement_resolution"])
    
    tg.run()
    print("run finished")
    tg.step2()
    print("step2 finished")
    del(tg)
    

0 / 66
101
50
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

/Users/mjirik/projects/teigen/teigen/geometry3d.py:15: RuntimeWarning: invalid value encountered in divide
  vector = length * vector / np.linalg.norm(vector)
/Users/mjirik/projects/teigen/teigen/geometry3d.py:342: RuntimeWarning: invalid value encountered in greater
  if np.all(node > (0 + radius)) and np.all(node < (areasize - radius)):
/Users/mjirik/projects/teigen/teigen/geometry3d.py:800: RuntimeWarning: invalid value encountered in divide
  vector /= np.linalg.norm(vector)



0.0
0.0
0.0
1.0
run finished
1.0
step2 finished
1 / 66
101
50
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
run finished
1.0
step2 finished
2 / 66
101
50
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
run finished
1.0
step2 finished
3 / 66
101
50
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
run finished
1.0
step2 finished
4 / 66
101
50
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
run finished
1.0
step2 finished
5 / 66
101
50
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
run finished
1.0
step2 finished
6 / 66
101
50
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
run finished
1.0
step2 finished
7 / 66
101
50
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
run finished
1.0
step2 finished
8 / 66
101
50
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
run finished
1.0
step2 finished
9 / 66
101
50
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0

/Users/mjirik/projects/teigen/teigen/tb_vtk.py:439: RuntimeWarning: invalid value encountered in divide
  br['direction'] = vv / nm.linalg.norm(vv)
